In [1]:
%matplotlib inline
import h5py
import numpy as np
import os
from UG2.utils import data as data_utils
from UG2.utils import image as image_utils
from UG2.config import Config
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import scipy.ndimage as im
import time
from UG2 import main

In [2]:
data_path            = "/data/UG2_data/h5Files/"
traindedModels_path  = "/data/UG2_data/trainedModels/"

In [3]:
## create blurred-dataset 
kernel_blurring_flag= True
num_images          = 100
factor              = 2
patch_size          = np.array([64, 64])
epochsList          = [700]
learningRateList    = [0.001]
numberTrainingKernel= [4]
flags               = [1,1]
numberOfExperiments = len(epochsList)*len(learningRateList)
dataset_name        = "BSDBlurred"

In [ ]:
trainStartIndex = 0
numberCompletedExperiments = 0
for i in range(trainStartIndex, len(epochsList)):
    for nLR in learningRateList:
        for nTK in numberTrainingKernel:
            config = Config() 
            config.epochs = epochsList[i]
            config.cuda = True
            config.data_parallel = True
            config.lr = nLR
            if kernel_blurring_flag:
                config.data_files = [os.path.join(data_path, dataset_name+"_nTK_"+str(nTK)+ "_flag_" + "_".join([str(f) for f in flags])+".h5")]
            else:        
                config.data_files = [os.path.join(data_path, dataset_name+".h5")]
            config.batch_size = 200
            config.print_step = 100
            config.checkpoint = 100
            config.exit_loss_window = 5
            config.loss_epsilon = 0.01
            config.resume_training_flag = False
            config.kernel_blurring_flag = False
            config.discriminator = "feat_ext"
            config.factor      = factor
            config.data_path   = data_path
            config.img_size    = 64
            config.num_workers = 8
            config.data_format = "h5"
            if i:
                config.epochs = epochsList[i]-epochsList[i-1]
                config.resume_training_flag = True
                config.resume_model_path = traindedModels_path
                config.resume_model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i-1])+".pth"
            
            config.model_path = traindedModels_path
            config.model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i])+"nTK"+str(nTK)+".pth"
            numberCompletedExperiments = numberCompletedExperiments+1
            main.train(config)
        
            print("Completed "+str(numberCompletedExperiments)+" out of "+str(numberOfExperiments))
        

time:  3.8892135620117188  Error:  5.78160007795
saved checkpoint at epoch:  0
time:  1.8912675380706787  Error:  0.799551288287
saved checkpoint at epoch:  100
time:  1.8801259994506836  Error:  0.610019783179
saved checkpoint at epoch:  200
time:  1.8905112743377686  Error:  0.543065845966
saved checkpoint at epoch:  300
time:  1.8833656311035156  Error:  0.490031987429
saved checkpoint at epoch:  400


In [ ]:
print(config.model_name)